## 准备数据

In [11]:
import os
import numpy as np
import torch
from torchvision.datasets import MNIST

## Demo numpy based auto differentiation

## Matmul

$$d\mathcal{L}=tr(\frac{\partial\mathcal{L}}{\partial y}^\top dy)=tr(\frac{\partial\mathcal{L}}{\partial y}^\top d(XW))=tr(\frac{\partial\mathcal{L}}{\partial y}^\top (dXW+XdW))=tr(\frac{\partial\mathcal{L}}{\partial y}^\top dXW)+tr(\frac{\partial\mathcal{L}}{\partial y}^\top XdW)=tr(W\frac{\partial\mathcal{L}}{\partial y}^\top dX)+tr((X^\top\frac{\partial\mathcal{L}}{\partial y})^\top dW)=tr((\frac{\partial\mathcal{L}}{\partial y}W^\top)^\top dX)+tr((X^\top\frac{\partial\mathcal{L}}{\partial y})^\top dW)$$

## Softmax
<!-- $$d\mathcal{L}=tr(\frac{\partial\mathcal{L}}{\partial y}^\top dy)=tr(\frac{\partial\mathcal{L}}{\partial y}^\top d\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top})=tr(\frac{\partial\mathcal{L}}{\partial y}^\top d\ (\exp(X)\odot \frac{1}{\exp(X)\mathbf{1}\mathbf{1}^\top}))$$ -->
$$dy=d\ Softmax(x)=d\ \frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top}=d\ \exp(X)\odot \frac{1}{\exp(X)\mathbf{1}\mathbf{1}^\top} = d\ (\exp(X)\odot \frac{1}{\exp(X)\mathbf{1}\mathbf{1}^\top})=(d\ \exp(X))\odot \frac{1}{\exp(X)\mathbf{1}\mathbf{1}^\top}+\exp(X)\odot d(\frac{1}{\exp(X)\mathbf{1}\mathbf{1}^\top})=(\exp(X)\odot dX)\odot \frac{1}{\exp(X)\mathbf{1}\mathbf{1}^\top}-\exp(X)\odot (\frac{1}{(\exp(X)\mathbf{1}\mathbf{1}^\top)^2}\odot((\exp(X)\odot dX)\mathbf{1}\mathbf{1}^\top))$$
$$d\mathcal{L}=tr(\frac{\partial\mathcal{L}}{\partial y}^\top dy)=tr(\frac{\partial\mathcal{L}}{\partial y}^\top((\exp(X)\odot dX)\odot \frac{1}{\exp(X)\mathbf{1}\mathbf{1}^\top}-\exp(X)\odot \frac{1}{(\exp(X)\mathbf{1}\mathbf{1}^\top)^2}\odot\exp(X)\odot dX\mathbf{1}\mathbf{1}^\top) )=tr((\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top}\odot\frac{\partial\mathcal{L}}{\partial y})^\top  dX-(\exp(X)\odot \frac{\partial\mathcal{L}}{\partial y})^\top (\frac{1}{(\exp(X)\mathbf{1}\mathbf{1}^\top)^2}((\exp(X)\odot dX)\mathbf{1}\mathbf{1}^\top))) $$
其中，（下面的推导中，括号是关键）
$$tr((\frac{\exp(X)}{(\exp(X)\mathbf{1}\mathbf{1}^\top)}\odot \frac{\partial\mathcal{L}}{\partial y})^\top (\frac{1}{(\exp(X)\mathbf{1}\mathbf{1}^\top)}((\exp(X)\odot dX)\mathbf{1}\mathbf{1}^\top)))=tr((\frac{\exp(X)}{(\exp(X)\mathbf{1}\mathbf{1}^\top)}\odot \frac{\partial\mathcal{L}}{\partial y})^\top ((\frac{\exp(X)}{(\exp(X)\mathbf{1}\mathbf{1}^\top)}\odot dX)\mathbf{1}\mathbf{1}^\top))=tr(\mathbf{1}\mathbf{1}^\top ((\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top}\odot \frac{\partial\mathcal{L}}{\partial y})^\top(\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top}\odot dX)))=tr(((\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top})^\top\odot(\mathbf{1}\mathbf{1}^\top (\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top}\odot \frac{\partial\mathcal{L}}{\partial y})^\top) dX))$$
从而有
<!-- $$d\mathcal{L}=tr((\frac{\exp(X)}{(\exp(X)\mathbf{1}\mathbf{1}^\top)}\odot \frac{\partial\mathcal{L}}{\partial y})^\top ((\frac{\exp(X)}{(\exp(X)\mathbf{1}\mathbf{1}^\top)}\odot dX)\mathbf{1}\mathbf{1}^\top-dX))$$ -->
<!-- $$tr(\exp(X)\odot \frac{1}{(\exp(X)\mathbf{1}\mathbf{1}^\top)^2}\odot\frac{\partial\mathcal{L}}{\partial y}^\top(\exp(X)\odot dX\mathbf{1}\mathbf{1}^\top))=tr((\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top})^2\odot\frac{\partial\mathcal{L}}{\partial y}^\top(dX\mathbf{1}\mathbf{1}^\top))=tr(dX\mathbf{1}\mathbf{1}^\top(\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top})^2\odot\frac{\partial\mathcal{L}}{\partial y}^\top)=tr(\mathbf{1}\mathbf{1}^\top(\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top})^2\odot \frac{\partial\mathcal{L}}{\partial y}^\top dX)$$ -->
$$\frac{\partial\mathcal{L}}{\partial X}=\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top}\odot\frac{\partial\mathcal{L}}{\partial y}-\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top}\odot( (\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top}\odot \frac{\partial\mathcal{L}}{\partial y})\mathbf{1}\mathbf{1}^\top)=\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top}\odot(\frac{\partial\mathcal{L}}{\partial y}- (\frac{\exp(X)}{\exp(X)\mathbf{1}\mathbf{1}^\top}\odot \frac{\partial\mathcal{L}}{\partial y})\mathbf{1}\mathbf{1}^\top )$$



In [12]:
import numpy as np
class Matmul:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x, W):
        h = np.matmul(x, W)
        self.mem={'x': x, 'W':W}
        return h
    
    def backward(self, grad_y):
        '''
        x: shape(N, d)
        w: shape(d, d')
        grad_y: shape(N, d')
        '''
        x = self.mem['x']
        W = self.mem['W']
        
        ####################
        '''计算矩阵乘法的对应的梯度'''
        ####################
        grad_x = np.matmul(grad_y,W.T)
        grad_W = np.matmul(x.T,grad_y)
        return grad_x, grad_W


In [13]:


class Relu:
    def __init__(self):
        self.mem = {}
        
    def forward(self, x):
        self.mem['x']=x
        return np.where(x > 0, x, np.zeros_like(x))
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        ####################
        '''计算relu 激活函数对应的梯度'''
        ####################
        x = self.mem['x']
        grad_x = np.where(x>0,grad_y,np.zeros_like(x))
        return grad_x
    



In [51]:
class Softmax:
    '''
    softmax over last dimension
    '''
    def __init__(self):
        self.epsilon = np.finfo(np.float64).eps
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        # 增加数值稳定性
        x = x - x.max(axis=1,keepdims=True)
        x_exp = np.exp(x)
        partition = np.sum(x_exp, axis=1, keepdims=True)
        out = x_exp/np.clip(partition,self.epsilon,None)
        
        self.mem['out'] = out
        self.mem['x_exp'] = x_exp
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        s = self.mem["out"]
        return s*(grad_y - (s*grad_y).sum(axis=1,keepdims=True))

In [15]:
    
class Log:
    '''
    softmax over last dimension
    '''
    def __init__(self):
        self.epsilon = np.finfo(np.float64).eps
        self.mem = {}
        
    def forward(self, x):
        '''
        x: shape(N, c)
        '''
        out = np.log(np.clip(x,self.epsilon,None))
        
        self.mem['x'] = x
        return out
    
    def backward(self, grad_y):
        '''
        grad_y: same shape as x
        '''
        x = self.mem['x']
        
        return 1./np.clip(x,self.epsilon,None) * grad_y
    

## Gradient check

In [16]:
import torch
import torch.nn.functional as F
import numpy as np

x = np.random.normal(size=[5, 6])
W = np.random.normal(size=[6, 4])
aa = Matmul()
out = aa.forward(x, W) # shape(5, 4)
grad_x,grad_W = aa.backward(np.ones_like(out))

x,W = torch.tensor(x,requires_grad=True),torch.tensor(W,requires_grad=True)
y = torch.matmul(x,W)
loss = y.sum()
loss.backward()
print(torch.equal(x.grad,torch.tensor(grad_x)))
print(torch.equal(W.grad,torch.tensor(grad_W)))



x = np.random.normal(size=[5, 6])
aa = Relu()
out = aa.forward(x) # shape(5, 4)
grad = aa.backward(np.ones_like(out))

x = torch.tensor(x,requires_grad=True)
y = F.relu(x)
loss = y.sum()
loss.backward()
print(torch.equal(x.grad,torch.tensor(grad)))

x = np.random.normal(size=[5, 6], scale=5.0, loc=1)
aa = Softmax()
out = aa.forward(x) # shape(5, 6)
grad = aa.backward(np.ones_like(out))

x = torch.tensor(x,requires_grad=True)
y = F.softmax(x,dim=1)
loss = y.sum()
loss.backward()
# 这里因为上游梯度都是1,所以其实得到的grad全是0，但是由于误差，所以不是全0
print(torch.allclose(x.grad,torch.tensor(grad)))

x = np.clip(np.random.normal(10,1,size=[5, 6]),0,None)
aa = Log()
out = aa.forward(x) # shape(5, 4)
grad = aa.backward(np.ones_like(out))

x = torch.tensor(x,requires_grad=True)
y = torch.log(x)
loss = y.sum()
loss.backward()
print(torch.allclose(x.grad,torch.tensor(grad)))

True
True
True
True
True


# Final Gradient Check

In [17]:
import torch
import torch.nn.functional as F


x = np.random.normal(size=[5, 6])
W1 = np.random.normal(size=[6, 5])
W2 = np.random.normal(size=[5, 6])

label = np.zeros_like(x)
label[0, 1]=1.
label[1, 0]=1
label[2, 3]=1
label[3, 5]=1
label[4, 0]=1

mul_h1 = Matmul()
mul_h2 = Matmul()
relu = Relu()
softmax = Softmax()
log = Log()

np_h1 = mul_h1.forward(x, W1) # shape(5, 4)
np_h1_relu = relu.forward(np_h1)
np_h2 = mul_h2.forward(np_h1_relu, W2)
np_h2_soft = softmax.forward(np_h2)
np_h2_log = log.forward(np_h2_soft)


h2_log_grad = log.backward(label)
h2_soft_grad = softmax.backward(h2_log_grad)
h2_grad, W2_grad = mul_h2.backward(h2_soft_grad)
h1_relu_grad = relu.backward(h2_grad)
h1_grad, W1_grad = mul_h1.backward(h1_relu_grad)


x = torch.tensor(x,requires_grad=True)
W1 = torch.tensor(W1,requires_grad=True)
W2 = torch.tensor(W2,requires_grad=True)
label = torch.tensor(label,requires_grad=True)
h1 = torch.matmul(x, W1)
h1.retain_grad()
h1_relu = F.relu(h1)
h1_relu.retain_grad()
h2 = torch.matmul(h1_relu, W2)
h2.retain_grad()
prob = F.softmax(h2,dim=1)
prob.retain_grad()
log_prob = torch.log(prob)
log_prob.retain_grad()
loss = (label * log_prob).sum()
loss.backward()

print("Forward check:")
print(torch.allclose(torch.tensor(np_h1),h1))
print(torch.allclose(torch.tensor(np_h1_relu),h1_relu))
print(torch.allclose(torch.tensor(np_h2),h2))
print(torch.allclose(torch.tensor(np_h2_soft),prob))
print(torch.allclose(torch.tensor(np_h2_log),log_prob))
print("Backward check:")

print(torch.allclose(torch.tensor(h2_log_grad),prob.grad))
print(torch.allclose(torch.tensor(h2_soft_grad),h2.grad))
print(torch.allclose(torch.tensor(W2_grad),W2.grad))
print(torch.allclose(torch.tensor(h2_grad),h1_relu.grad))
print(torch.allclose(torch.tensor(h1_relu_grad),h1.grad))
print(torch.allclose(torch.tensor(h1_grad),x.grad))
print(torch.allclose(torch.tensor(W1_grad),W1.grad))

Forward check:
True
True
True
True
True
Backward check:
True
True
True
True
True
True
True


## 建立模型

In [54]:
class myModel:
    def __init__(self):
        
        self.W1 = np.random.normal(0,1,size=[28*28+1, 100])
        self.W2 = np.random.normal(0,1,size=[100, 10])
        
        self.mul_h1 = Matmul()
        self.mul_h2 = Matmul()
        self.relu = Relu()
        self.softmax = Softmax()
        self.log = Log()
        
        
    def forward(self, x):
        x = x.reshape(-1, 28*28)
        bias = np.ones(shape=[x.shape[0], 1])
        x = np.concatenate([x, bias], axis=1)
        
        self.h1 = self.mul_h1.forward(x, self.W1) # shape(5, 4)
        self.h1_relu = self.relu.forward(self.h1)
        self.h2 = self.mul_h2.forward(self.h1_relu, self.W2)
        self.h2_soft = self.softmax.forward(self.h2)
        self.h2_log = self.log.forward(self.h2_soft)
            
    def backward(self, label):
        self.h2_log_grad = self.log.backward(-label)
        self.h2_soft_grad = self.softmax.backward(self.h2_log_grad)
        self.h2_grad, self.W2_grad = self.mul_h2.backward(self.h2_soft_grad)
        self.h1_relu_grad = self.relu.backward(self.h2_grad)
        self.h1_grad, self.W1_grad = self.mul_h1.backward(self.h1_relu_grad)
        


## 计算 loss

In [45]:
def compute_loss(log_prob, labels):
     return np.mean(np.sum(-log_prob*labels, axis=1))
    

def compute_accuracy(log_prob, labels):
    predictions = np.argmax(log_prob, axis=1)
    truth = np.argmax(labels, axis=1)
    return np.mean(predictions==truth)

def train_one_step(model, x, y):
    model.forward(x)
    model.backward(y)
    model.W1 -= 1e-5* model.W1_grad
    model.W2 -= 1e-5* model.W2_grad
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

def test(model, x, y):
    model.forward(x)
    loss = compute_loss(model.h2_log, y)
    accuracy = compute_accuracy(model.h2_log, y)
    return loss, accuracy

def mnist_dataset(train_ratio=0.8):
    mnist = MNIST(root=".",download=True)
    data,targets = mnist.data.numpy()/255,mnist.targets.numpy()
    data = data.reshape(data.shape[0],-1)
    data = (data-data.mean(axis=1,keepdims=True))/data.std(axis=1,keepdims=True)
    idx = np.arange(data.shape[0])
    train_idx = np.zeros_like(idx)
    train_idx[np.random.choice(idx,int(train_ratio*data.shape[0]),replace=False)] = 1
    train_x,train_y = data[train_idx.astype(bool)],targets[train_idx.astype(bool)]
    test_x,test_y = data[train_idx.astype(bool)],targets[train_idx.astype(bool)]
    return train_x,train_y,test_x,test_y

## 实际训练

In [55]:
train_x,train_y,test_x,test_y = mnist_dataset()
train_x,test_x = train_x,test_x
train_label,test_label = np.zeros((train_x.shape[0],10)),np.zeros((test_x.shape[0],10))
train_label[np.arange(train_x.shape[0]),train_y] = 1
test_label[np.arange(test_x.shape[0]),test_y] = 1
model = myModel()
for epoch in range(50):
    loss, accuracy = train_one_step(model,train_x , train_label)
    print('epoch', epoch, ': loss', loss, '; accuracy', accuracy)
loss, accuracy = test(model, test_x, test_label)

print('test loss', loss, '; accuracy', accuracy)

epoch 0 : loss 30.631913034555094 ; accuracy 0.12839583333333332
epoch 1 : loss 27.072987381655704 ; accuracy 0.23035416666666667
epoch 2 : loss 25.61888859695632 ; accuracy 0.26747916666666666
epoch 3 : loss 24.21164729959287 ; accuracy 0.3082708333333333
epoch 4 : loss 22.887691538580192 ; accuracy 0.34477083333333336
epoch 5 : loss 21.468690855442556 ; accuracy 0.3829166666666667
epoch 6 : loss 19.855945958501845 ; accuracy 0.42752083333333335
epoch 7 : loss 18.842740373466903 ; accuracy 0.45660416666666664
epoch 8 : loss 18.10483127172347 ; accuracy 0.47860416666666666
epoch 9 : loss 17.567477405436552 ; accuracy 0.49364583333333334
epoch 10 : loss 17.09499048997183 ; accuracy 0.5077083333333333
epoch 11 : loss 16.687981868367526 ; accuracy 0.5185
epoch 12 : loss 16.269297402354542 ; accuracy 0.5302708333333334
epoch 13 : loss 15.737395866555154 ; accuracy 0.544125
epoch 14 : loss 15.02529080012623 ; accuracy 0.561875
epoch 15 : loss 14.677868970818185 ; accuracy 0.5715208333333334